In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pickle as pkl
import os
import sys
import jax.tree_util as jtu
import subprocess

In [2]:
sys.path.append(os.path.dirname(os.getcwd()))
from ml_collections import ConfigDict
from rdkit import Chem
from onescience.flax_models.MolSculptor.train.scheduler import GaussianDiffusion
from onescience.flax_models.MolSculptor.train.inference import smi2graph_features
from onescience.flax_models.MolSculptor.train.rewards import smi2pdbqt, dsdp_reward

In [3]:
lead_smi = 'CCOC(=O)c1ccc(NC(=O)c2cccc(S(=O)(=O)N3CCCc4ccccc43)c2)s1'
lead_smi = Chem.MolToSmiles(Chem.MolFromSmiles(lead_smi), canonical=True)
print("Canonical SMILES: ", lead_smi)

Canonical SMILES:  CCOC(=O)c1ccc(NC(=O)c2cccc(S(=O)(=O)N3CCCc4ccccc43)c2)s1


In [10]:
def dsdp_reward(smi: str, cached_file_path: str, dsdp_script_path: str):
    smi_save_dir = os.path.join(cached_file_path, f'temp-ligand.pdbqt')
    smi2pdbqt(smi, smi_save_dir)
    out_dir = os.path.join(cached_file_path, f'temp-dock.pdbqt')
    log_dir = os.path.join(cached_file_path, f'temp-log.log')
    cmd = ['bash', dsdp_script_path, smi_save_dir, out_dir, log_dir]
    cmd = ' '.join(cmd)
    subprocess.run(cmd, check=True, shell=True)
    with open(log_dir, 'r') as f:
        lines = f.readlines()
        ss = [float(s.split()[-1]) for s in lines]
    return ss[0]

In [ ]:
os.makedirs('./init-molecule', exist_ok=True)
root_wd = os.path.dirname(os.getcwd())
graphs = smi2graph_features(lead_smi)
score = [
    dsdp_reward(lead_smi, f'{root_wd}/tests/init-molecule', f'{root_wd}/cases/case_ar-gr/dsdp_ar.sh'),
    dsdp_reward(lead_smi, f'{root_wd}/tests/init-molecule', f'{root_wd}/cases/case_ar-gr/dsdp_gr.sh'),
]
score = np.asarray(score, np.float32) * (-1)
print(score)

/lustre/grp/gyqlab/liyh/stable_moledit/MolSculptor/tests/init-molecule/temp-ligand.pdbqt
Total time 2.376494 s
/lustre/grp/gyqlab/liyh/stable_moledit/MolSculptor/tests/init-molecule/temp-ligand.pdbqt
Total time 2.331549 s


In [13]:
molecule = {
        'scores': np.asarray(score, np.float32),
        'smiles': np.asarray(lead_smi, object),
        'graphs': graphs,
    }

In [14]:
with open(f'./init-molecule/init_search_molecule.pkl', 'wb') as f:
    pkl.dump(molecule, f)